In [1]:

## versions:
## Python    : 3.11.5
## numpy     : 1.26.0
## torch     : 2.1.0
## pandas    : 2.1.1

# licensed under the Creative Commons - Attribution-NonCommercial 4.0
# International license (CC BY-NC 4.0):
# https://creativecommons.org/licenses/by-nc/4.0/. 

import os
import io
import sys
import shutil
import datetime
from typing import Dict, List, Optional
from copy import deepcopy

import numpy as np
import pandas as pd
import torch as t
from torch.utils.data import DataLoader
from scipy import stats
import matplotlib.pyplot as plt

from common.torch.ops import empty_gpu_cache
from common.sampler import ts_dataset
from common.torch.snapshots import SnapshotManager
from experiments.trainer import trainer_var
from experiments.model import generic_dec_var
from models.exog import TCN_encoder

from data_utils.forecast import tryJSON, Struct, read_config, default_settings, make_training_fn
from data_utils.forecast import init_target_data, load_exog_data, make_training_fn, generate_quantiles
from data_utils.forecast import pickle_results, read_pickle, output_figs
from data_utils.flu import domain_defaults, specify_ensemble, output_df, append_forecasts
from data_utils.flu import read_flu_data, read_weather_data


In [2]:
import warnings
%config InlineBackend.figure_formats = ["svg"]
plt.style.use("dark_background")
warnings.formatwarning = lambda message, category, *args, **kwargs: "{}: {}\n".format(category.__name__, message)
warnings.filterwarnings("ignore",category=FutureWarning)
#%load_ext watermark
#%watermark -n -u -v -iv -w

(if needed) read latest data

In [ ]:
#idx, _ = read_flu_data()
#read_weather_data(idx)

`read_config()` returns configuration settings that don't change between models within an ensemble

gets values from `config.json` if available

see comments in `data_utils/forecast.py` for an explanation of entries

In [3]:
rstate = read_config("config_flu.json")

In [ ]:
rstate

you can change the settings here or in `config.json`

e.g., `rstate.cut` sets the train/test split index (None = train on all data)

In [5]:
rstate.cut = None #420 #447 #395 #420 #425 #430 

`default_settings()` returns settings that can be changed between models within an ensemble

gets defaults from `settings.json` if available

see comments in `data_utils/forecast.py` for an explanation of entries

In [6]:
settings = default_settings("settings_flu.json")

can change settings in json file or here

In [7]:
#try increasing the learning rate when there's more training data
#settings.init_LR = np.round(0.0001 + (rstate.cut - 901) * 4e-7, 7) 

we will change `settings.exog_vars` below, to specify which exogenous predictors to use

In [ ]:
settings

`domain_defaults()` is meant to be a user-defined function

returns a struct with instructions for reading or generating exogenous variables

see `data_utils/covid_hub.py` for an example/explanation

In [8]:
domain_specs = domain_defaults()

`exog_vars` specifies which exogenous predictors to use by default

the predictors in `var_names` are loaded/generated and available to use

In [ ]:
domain_specs

`init_target_data()` reads in and optionally transforms target data

sets timepoint indices and series identifiers; writes data to `rstate`


In [10]:
rstate, settings = init_target_data(rstate, settings)

`rstate.data_index` was set based on the index of `rstate.target_file`

for exogenous data, the files and functions specified in `domain_defaults()` must generate data frames with the same index

In [ ]:
rstate.data_dir+"/"+rstate.target_file, rstate.data_index

`load_exog_data()` appends exogenous predictors to rstate, using the data index generated above

In [12]:
rstate, settings = load_exog_data(rstate, settings, domain_specs)

`settings.exog_vars` now has the defaults from domain_specs (if this was not set in `settings.json`)

In [ ]:
settings.exog_vars

the data has been read into `rstate` as a dict keyed by series name

each series is a data frame with rows as timepoints and columns as variables

In [ ]:
rstate.series_dfs["Maryland"]

the name of the target column was set automatically by `init_target_data()`

In [ ]:
rstate.target_var

`make_training_fn()` returns a function that trains a model  (it closes over training data and config settings)

the resulting function takes `settings` and returns mean & variance forecasts

the forecasts are matrices with rows = series and columns = timepoints

the trained models are saved in `rstate.snapshot_dir`

the training function can be used on its own or called in a loop with different settings to generate an ensemble


In [16]:
training_fn = make_training_fn(rstate)

to train an ensemble of models, we will generate a list of `settings`, one for each model

`specify_ensemble` is a user-defined function that generates the list, based on info in `domain_specs`

see `data_utils/covid_hub.py` for an example

In [17]:
## maybe we don't really need 5 random reps
domain_specs.random_reps = 3

## generate a list of settings structs having the desired variation for ensemble
## save the list to rstate for posterity
rstate.settings_list = specify_ensemble(settings, domain_specs)


can also define some other ensemble:

In [3]:
## setting size of hidden layer based on size of lookback window:
def custom_ensemble(template, specs):
    settings_list = []
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            x = deepcopy(template)
            x.lookback = opt
            x.nbeats_hidden_dim = opt * 2 * 6 * 8
            settings_list.append(x)
    return settings_list


to use snapshot/pretrained model with no additional training, set iterations to 0

In [19]:
## use snapshot/pretrained model, no additional training
settings.iterations = 0

In [ ]:
rstate.settings_list = custom_ensemble(settings, domain_specs)
rstate.settings_list[3]

(optional) a pretrained model file for each model in the ensemble

each must have the same structure (lookback window, hidden dims, etc.) as the corresponding ensemble entry

In [4]:

def pretrained_list(pretrain_dir, specs):
    file_list = []
    for j in range(specs.random_reps):
        for opt in specs.lookback_opts:
            filename = "flu24_" + str(opt) + "H_" + str(j+1) + ".pt"
            file_list.append(os.path.join(pretrain_dir,filename))
    return file_list


In [ ]:
rstate.pretrained_models = [None for x in rstate.settings_list]

pretrain_dir = "flu_pretrained_2024" # None # 

if pretrain_dir is not None:
    rstate.pretrained_models = pretrained_list(os.path.join(rstate.data_dir,pretrain_dir), domain_specs)

rstate.pretrained_models

empty dicts for storing the forecasts from each model:

In [23]:
mu_fc={}
var_fc={}

In [24]:
empty_gpu_cache() ## just in case?

train each model in the ensemble and write its forecast to `mu_fc` and `var_fc` (keyed w/ a semi-descriptive name):

In [ ]:

## ensemble loop
for i, set_i in enumerate(rstate.settings_list):
    model_name = rstate.output_prefix+"_"+str(i)
    model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
    model_name = model_name+"_"+model_suffix
    print("training ",model_name)
    mu_fc[model_name], var_fc[model_name] = training_fn(model_name, set_i, rstate.pretrained_models[i]) 


forecast shape for each model is [series, time]

ensemble the dict values using median across models

write results to `rstate`

In [26]:

mu_fc["ensemble"] = np.median(np.stack([mu_fc[k] for k in mu_fc]),axis=0)
var_fc["ensemble"] = np.median(np.stack([var_fc[k] for k in var_fc]),axis=0)

rstate.mu_fc = mu_fc
rstate.var_fc = var_fc


if forecast targets are per-capita, need series weights for summing to national (per capita) forecast

In [ ]:
if rstate.series_weights is not None:
    print(pd.DataFrame({"state":rstate.series_names,"weight":rstate.series_weights.squeeze()}))

`generate_quantiles()` goes through each entry in `rstate.mu_fc` and `rstate.var_fc`

and generates dicts containing forecast quantiles for each model (and "ensemble")

see comments in `data_utils/forecast.py` for details

In [28]:
rstate = generate_quantiles(rstate)

optional: save rstate, which contains all training data, forecasts, and ensemble settings

`pickle_results()` writes it to output dir

In [29]:
pickle_results(rstate)

plot some forecasts

In [ ]:
output_figs(rstate, rstate.settings_list[0].horizon, 
#[0,1,2,4], 
#[8,9,10,11], 
range(11),
 70,
 colors=["white","yellow"],figsize=(5,3),plot_mean=True)

save forecasts as csv

In [ ]:
df,_ = output_df(rstate,0)
df.query("location == 'US' and quantile=='mean'")


In [40]:
append_forecasts(df, os.path.join(rstate.data_dir,"forecast_plots.csv"))


delete the trained models if we no longer need them:

In [28]:
if rstate.delete_models:
    try:
        shutil.rmtree(rstate.snapshot_dir)
    except:
        pass


automate the above

In [5]:
def init_rstate(cut, settings, domain_specs, ensemble_fn=specify_ensemble):
    rstate = read_config("config_flu.json")
    rstate.cut = cut
    
    rstate, settings = init_target_data(rstate, settings)
    rstate, settings = load_exog_data(rstate, settings, domain_specs)
    rstate.settings_list = ensemble_fn(settings, domain_specs)
    rstate.pretrained_models = [None for x in rstate.settings_list]
    
    return rstate, settings


def generate_ensemble(rstate):
    mu_fc={}
    var_fc={}
    empty_gpu_cache()
    training_fn = make_training_fn(rstate)

    ## ensemble loop
    for i, set_i in enumerate(rstate.settings_list):
        model_name = rstate.output_prefix+"_"+str(i)
        model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
        model_name = model_name+"_"+model_suffix
        print("training ",model_name)
        mu_fc[model_name], var_fc[model_name] = training_fn(model_name, set_i, rstate.pretrained_models[i]) 

    mu_fc["ensemble"] = np.median(np.stack([mu_fc[k] for k in mu_fc]),axis=0)
    var_fc["ensemble"] = np.median(np.stack([var_fc[k] for k in var_fc]),axis=0)
    rstate.mu_fc = mu_fc
    rstate.var_fc = var_fc

    rstate = generate_quantiles(rstate)

    return rstate


def delete_model_dir(rstate):
    if rstate.delete_models:
        try:
            shutil.rmtree(rstate.snapshot_dir)
        except:
            pass


In [6]:

## try adjusting the amount of training based on the amount of training data history
## (lowering learning rate seems to work better than decreasing # of iterations)
def adapt_iter(x):
    return None#int(np.round(200 + (x - 901) * 2.0 / 3.0))

def adapt_lr(x):
    return None#np.round(0.0001 + (x - 901) * 4e-7, 7) 

def run_test(cut, random_reps=None, ensemble_fn=specify_ensemble, series_figs=[], n_iter=None, pretrain_dir=None, adj_iter=False, adj_LR=False):
    ## if adj_*, train more when there is more data; otherwise use values from settings.json
    settings = default_settings("settings_flu.json")
    #if adj_iter: settings.iterations = adapt_iter(cut)
    #if adj_LR: settings.init_LR = adapt_lr(cut)
    if n_iter is not None: settings.iterations = n_iter

    domain_specs = domain_defaults()
    if random_reps is not None: domain_specs.random_reps = random_reps
    
    rstate, settings = init_rstate(cut, settings, domain_specs, ensemble_fn)

    if pretrain_dir is not None:
        rstate.pretrained_models = pretrained_list(os.path.join(rstate.data_dir,pretrain_dir), domain_specs)

    rstate = generate_ensemble(rstate)

    pickle_results(rstate)
    output_figs(rstate, rstate.settings_list[0].horizon, 
                series_figs, 
                70,
                colors=["white","yellow"],figsize=(5,3),plot_mean=True)

    df,_ = output_df(rstate,0)
    append_forecasts(df, os.path.join(rstate.data_dir,"forecast_plots.csv"))

    delete_model_dir(rstate)


graph training losses

note, ensembling not-quite-converged models seems to work better than running more iterations


In [36]:

def plot_losses(pickle_file,ylim=None):
    rstate = read_pickle(pickle_file)
    model_prefix = rstate.output_prefix
    model_suffix = str(rstate.cut) if rstate.cut is not None else str(rstate.data_index[-1])
    _, ax = plt.subplots(nrows=len(rstate.settings_list),ncols=2,figsize=[8,2*len(rstate.settings_list)])
    for i, set_i in enumerate(rstate.settings_list):
        model_name =  model_prefix+"_"+str(i)+"_"+model_suffix
        total_iter = set_i.iterations
        snapshot_manager = SnapshotManager(snapshot_dir=os.path.join(rstate.snapshot_dir, model_name), total_iterations=total_iter)
        ldf = snapshot_manager.load_training_losses()
        vdf = snapshot_manager.load_validation_losses()
        ax[i,0].plot(ldf)
        ax[i,1].plot(vdf)
        ax[i,1].set_ylim(ylim)
    #plt.show()
    plt.savefig(os.path.join(rstate.output_dir , "losses_"+model_prefix+"_"+model_suffix+".png"))


In [ ]:
plot_losses(os.path.join(rstate.output_dir, "flu_420.pickle"))

In [ ]:
run_test(420, 3, custom_ensemble, series_figs=range(12), n_iter=0, pretrain_dir="flu_pretrained_2023")

In [ ]:
for cut in [467, 468, 469, 470, None]:
    run_test(cut, 3, custom_ensemble, series_figs=[], n_iter=0, pretrain_dir="flu_pretrained_2024")

In [ ]:
#rstate.delete_models = True
#delete_model_dir(rstate)